In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
import os
import seaborn as sns
from datetime import datetime, timedelta

In [2]:
animaldf = pd.read_csv("animal_info.csv", sep = ";")
animaldf.columns = [col.replace(" ", "_").lower().replace("-", "") for col in animaldf.columns]
fresh_dates = animaldf[["animal_number", "last_calving_date"]]
fresh_dates["fresh_date"] = pd.to_datetime(fresh_dates["last_calving_date"], format = '%m/%d/%Y')
fresh_dates = fresh_dates[~fresh_dates.fresh_date.isnull()]
fresh_dates = fresh_dates.drop("last_calving_date", axis = 1)
fresh_dates = fresh_dates[(~fresh_dates.animal_number.isna())]

df = pd.read_csv(r"more_reports\oneyear_data.csv", sep = ";")
df.columns = [col.replace(" ", "_").lower().replace("-", "") for col in df.columns]
df["date"] = pd.to_datetime(df["date"], format = '%m/%d/%Y')
merged_df =  pd.merge(fresh_dates, df,  left_on="animal_number", right_on = "animal_number")

merged_df["dim"] = (merged_df["date"] - merged_df["fresh_date"]).dt.days
df2 = merged_df[merged_df.dim >= 0]

C:\Users\marie\AppData\Local\Temp\ipykernel_17484\224687448.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fresh_dates["fresh_date"] = pd.to_datetime(fresh_dates["last_calving_date"], format = '%m/%d/%Y')


In [3]:
pf_df = df2[['animal_number', 'peak_flow_lf', 'peak_flow_lr', 'peak_flow_rf', 'peak_flow_rr', 'dim']]
pf_gdf = pf_df.groupby(['animal_number']).agg({'peak_flow_lf': 'mean', 'peak_flow_lr':'mean', 'peak_flow_rf': 'mean', 'peak_flow_rr': 'mean', 'dim':'max'}).reset_index()
pf_gdf = pf_gdf.rename(columns = {'peak_flow_lf' : 'peak_flow_lf_cl', 'peak_flow_rf':'peak_flow_rf_cl', 'peak_flow_rr':'peak_flow_rr_cl', 'peak_flow_lr': 'peak_flow_lr_cl'})

pf1 = pf_gdf

In [4]:
pf_df = df2[(df2.date >= '2023-11-10') & (df2.date < '2023-11-17')]
pf_df = pf_df[['animal_number', 'peak_flow_lf', 'peak_flow_lr', 'peak_flow_rf', 'peak_flow_rr', 'dim']]
pf_gdf = pf_df.groupby(['animal_number']).agg({'peak_flow_lf': 'mean', 'peak_flow_lr':'mean', 'peak_flow_rf': 'mean', 'peak_flow_rr': 'mean'}).reset_index()
pf_gdf = pf_gdf.rename(columns = {'peak_flow_lf' : 'peak_flow_lf_7d', 'peak_flow_rf':'peak_flow_rf_7d', 'peak_flow_rr':'peak_flow_rr_7d', 'peak_flow_lr': 'peak_flow_lr_7d'})
pf2 = pf_gdf

In [5]:
integrated_df = pf1.merge(pf2, how='left', on = "animal_number")
integrated_df.columns = [col.replace("rr", "rb").replace("lr", "lb").replace("animal_number", "cow") for col in integrated_df.columns]
integrated_df.to_csv(r"data_out\pf_integrated.csv", index = False)